**This short notebook is for the reference of the ring radius calculation based a theoretical formula provided by the TRIUMF team**

**IMPORTANT ASSUMPTION**: Since the theoretical formula to calculate ring radius is based on the mass of the particle and track momentum, we assume the label of particles in the event dataset is the ground truth.

In [1]:
import h5py 
import numpy as np 
import pandas as pd
import os
import seaborn as sns
import matplotlib.pyplot as plt
import sys
import glob
import warnings

In [2]:
# to keep the notebook neat
# import functions from a local py file 
# functions originally from https://github.com/TRIUMF-MDS-Capstone2023/resources
from helpers import events_to_pandas
from helpers import calc_ring_radius

In [3]:
# load the h5py raw data provided by the TRIUMF team for MDS 2023 capstone project
f = h5py.File('data/CaloRICH_Run11100_CTRL_v1.h5', 'r')
f

<HDF5 file "CaloRICH_Run11100_CTRL_v1.h5" (mode r)>

In [4]:
datasets = list(f.keys())
datasets

['Events', 'HitMapping', 'Hits']

In [5]:
# read the event dataset to pandas df
df = events_to_pandas(f)
df.head()

,run_id,burst_id,event_id,track_id,track_momentum,chod_time,track_pos_x,track_pos_y,ring_radius,ring_centre_pos_x,ring_centre_pos_y,ring_likelihood_pion,ring_likelihood_muon,ring_likelihood_positron,label,first_hit,last_hit,total_hits
0,11100,1468,11235,0,22.761024,7.789327,-195.126602,-153.930786,172.410950,-190.112167,-154.579514,2.083958e-12,1.0,1.914225e-24,0,0,20,20
1,11100,1468,11812,0,23.600529,0.000198,-316.197571,-57.275291,175.251694,-309.305939,-54.844410,7.732010e-10,1.0,8.275686e-19,0,20,39,19
2,11100,1468,14104,0,16.275131,11.789481,-88.681786,58.657421,155.040802,-88.665840,59.053833,1.216099e-37,1.0,1.216099e-37,0,39,65,26
3,11100,1468,14634,1,36.436443,7.426493,-39.124882,81.853058,185.832642,-35.864372,80.708580,8.006046e-03,1.0,3.626212e-02,0,65,117,52
4,11100,1468,18030,0,16.525362,8.923427,-66.697784,-15.932317,158.641846,-65.450981,-20.469883,1.216099e-37,1.0,1.216099e-37,0,117,141,24


In [7]:
# count the number of three particles
df['label'].value_counts()

0    2160219
1     215955
2      28515
Name: label, dtype: int64

In [8]:
# drop positron with label #2 as this is not our interest at this time
df = df.drop(df[df.label == 2].index)
df['label'].value_counts()

0    2160219
1     215955
Name: label, dtype: int64

In [10]:
# add new column with the corresponding mass value for pion and muon
# refer to calc_ring_radius() function provided in TRIUMF repo for details
# mass of the particle in MeV/c^2
df['mass'] = [105.66 if x == 0 else 139.57 for x in df['label']]

In [11]:
# add new column with the calculated ring radius from the theoretical formula
# ring_radius_cal in mm (consistent with ring_radius in the event dataset)
# in below code, the first 1000 is to convert track_momentum from GeV/c to MeV/c
# the second 1000 is to convert ring_radius_cal from m to mm
df['ring_radius_cal'] = calc_ring_radius(df['mass'], df['track_momentum']*1000)*1000

/Users/pengzh/opt/miniconda3/envs/591/lib/python3.10/site-packages/pandas/core/arraylike.py:402: RuntimeWarning: invalid value encountered in sqrt
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [12]:
df.head()

,run_id,burst_id,event_id,track_id,track_momentum,chod_time,track_pos_x,track_pos_y,ring_radius,ring_centre_pos_x,ring_centre_pos_y,ring_likelihood_pion,ring_likelihood_muon,ring_likelihood_positron,label,first_hit,last_hit,total_hits,mass,ring_radius_cal
0,11100,1468,11235,0,22.761024,7.789327,-195.126602,-153.930786,172.410950,-190.112167,-154.579514,2.083958e-12,1.0,1.914225e-24,0,0,20,20,105.66,173.437630
1,11100,1468,11812,0,23.600529,0.000198,-316.197571,-57.275291,175.251694,-309.305939,-54.844410,7.732010e-10,1.0,8.275686e-19,0,20,39,19,105.66,174.649395
2,11100,1468,14104,0,16.275131,11.789481,-88.681786,58.657421,155.040802,-88.665840,59.053833,1.216099e-37,1.0,1.216099e-37,0,39,65,26,105.66,155.334598
3,11100,1468,14634,1,36.436443,7.426493,-39.124882,81.853058,185.832642,-35.864372,80.708580,8.006046e-03,1.0,3.626212e-02,0,65,117,52,105.66,184.053395
4,11100,1468,18030,0,16.525362,8.923427,-66.697784,-15.932317,158.641846,-65.450981,-20.469883,1.216099e-37,1.0,1.216099e-37,0,117,141,24,105.66,156.484180


In [23]:
# output relevant columns from comparison
# assuming ring_radius was obtained by current NA62 MLE algorithm
df_r = df[['track_momentum',
           'mass',
           'ring_radius', 
           'ring_radius_cal',
           'label']]
df_r

,track_momentum,mass,ring_radius,ring_radius_cal,label
0,22.761024,105.66,172.410950,173.437630,0
1,23.600529,105.66,175.251694,174.649395,0
2,16.275131,105.66,155.040802,155.334598,0
3,36.436443,105.66,185.832642,184.053395,0
4,16.525362,105.66,158.641846,156.484180,0
...,...,...,...,...,...
2376169,31.104301,139.57,175.371704,174.599838,1
2376170,32.657063,139.57,178.343094,176.172961,1
2376171,19.860395,139.57,148.395798,148.473357,1
2376172,29.285913,139.57,175.335617,172.486949,1


In [25]:
df_r[['ring_radius', 'ring_radius_cal']].describe()

,ring_radius,ring_radius_cal
count,2.376161e+06,2.375492e+06
mean,7.081047e+03,1.797113e+02
std,8.450741e+05,1.205704e+01
min,0.000000e+00,3.912824e+00
25%,1.786680e+02,1.782955e+02
50%,1.840452e+02,1.835700e+02
75%,1.873861e+02,1.863779e+02
max,1.124473e+09,1.890850e+02


In [29]:
# check NaN results from the calculation of ring_radius_cal
# suspect the NaN is caused by floating issue covered by Lecture 1 of MDS DSCI 572.
# since all the NaN results are from lower momentum (max 12.4 GeV/c) that is out of [20,45]
# so we may drop all these rows (do need to dig more on NaN issue)
df_r_null = df_r.query('ring_radius_cal.isnull()')
df_r_null

,track_momentum,mass,ring_radius,ring_radius_cal,label
8287,7.696942,105.66,999999.000000,NaN,0
21595,9.248626,105.66,999999.000000,NaN,0
29788,7.355201,105.66,999999.000000,NaN,0
35723,8.642904,105.66,999999.000000,NaN,0
39546,8.917467,105.66,999999.000000,NaN,0
...,...,...,...,...,...
2371006,11.564781,139.57,999999.000000,NaN,1
2374410,9.270171,139.57,198.174637,NaN,1
2374667,11.472666,139.57,104.329933,NaN,1
2374875,11.416118,139.57,999999.000000,NaN,1


In [30]:
# mainly to check the track_momentum range for the NaN ring_radius_cal as explained above
df_r_null.describe()

,track_momentum,mass,ring_radius,ring_radius_cal,label
count,682.000000,682.000000,682.00000,0.0,682.000000
mean,8.730843,117.443974,756628.00000,NaN,0.347507
std,1.732889,16.159065,429396.28125,NaN,0.476528
min,4.585458,105.660000,0.00000,NaN,0.000000
25%,7.531076,105.660000,999999.00000,NaN,0.000000
50%,8.551003,105.660000,999999.00000,NaN,0.000000
75%,9.544966,139.570000,999999.00000,NaN,1.000000
max,12.432874,139.570000,999999.00000,NaN,1.000000
